In [ ]:
#Install bert package for tensorflow v1
!pip install bert-tensorflow==1.0.1
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

from datetime import datetime
import keras
from keras import layers
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tqdm.notebook import tqdm #adds progress bars to show loop status
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

class BERTClasiffier(object):

    """This class consists of functions to create, build, intialize and train the fine-tuned bert model as well as evaluate the model and 
    make predictions on a test set.
    Order of execution from top to bottom -
    GetPrediction -> evaluate -> train -> InitializeModel(internally called by train function) -> ModelFnBuilder (internally called
    by InitializeModel function) -> CreateModel (internally called by ModelFnBuilder function)"""

    def __init__(self, data, config, input):
        self.data = data
        self.config = config
        self.input = input

    def CreateModel(self, is_predicting, input_ids, input_mask, segment_ids, labels,
                     num_labels, BERT_MODEL_HUB):
  
        """Create the base model and add fine-tuning layers to the model"""

        bert_module = hub.Module(
            BERT_MODEL_HUB,
            trainable=True)
        bert_inputs = dict(
            input_ids=input_ids,
            input_mask=input_mask,
            segment_ids=segment_ids)
        bert_outputs = bert_module(
            inputs=bert_inputs,
            signature="tokens",
            as_dict=True)

        # Use "pooled_output" for classification tasks on an entire sentence.
        # Use "sequence_outputs" for token-level output.
        output_layer = bert_outputs["pooled_output"]
        # with tf.Session() as sess:
        output_layer1 = bert_outputs["pooled_output"]
        # output_layer1 = 999
        hidden_size = output_layer.shape[-1].value

        # Create our own layer to tune for data.
        output_weights = tf.get_variable(
            "output_weights", [num_labels, hidden_size],
            initializer=tf.truncated_normal_initializer(stddev=0.02))

        output_bias = tf.get_variable(
            "output_bias", [num_labels], initializer=tf.zeros_initializer())

        with tf.variable_scope("loss"):

            # Dropout helps prevent overfitting
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.8)

            logits = tf.matmul(output_layer, output_weights, transpose_b=True)
            logits = tf.nn.bias_add(logits, output_bias)
            log_probs = tf.nn.log_softmax(logits, axis=-1)

            # Convert labels into one-hot encoding
            one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

            predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
            # If we're predicting, we want predicted labels and the probabiltiies.
            if is_predicting:
                return (predicted_labels, log_probs, output_layer1)

            # If we're train/eval, compute loss between predicted and actual label
            per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
            loss = tf.reduce_mean(per_example_loss)
            return (loss, predicted_labels, log_probs)

    def ModelFnBuilder(self, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, BERT_MODEL_HUB):
        """Returns `model_fn` closure for TPUEstimator."""
        def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
            """The `model_fn` for TPUEstimator."""

            input_ids = features["input_ids"]
            input_mask = features["input_mask"]
            segment_ids = features["segment_ids"]
            label_ids = features["label_ids"]

            is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
            # TRAIN and EVAL
            if not is_predicting:

                (loss, predicted_labels, log_probs) = self.CreateModel(
                    is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels, BERT_MODEL_HUB)

                train_op = bert.optimization.create_optimizer(
                    loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

                # Calculate evaluation metrics. 
                def metric_fn(label_ids, predicted_labels):
                    accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                    true_pos = tf.metrics.true_positives(
                            label_ids,
                            predicted_labels)
                    true_neg = tf.metrics.true_negatives(
                            label_ids,
                            predicted_labels)   
                    false_pos = tf.metrics.false_positives(
                            label_ids,
                            predicted_labels)  
                    false_neg = tf.metrics.false_negatives(
                            label_ids,
                            predicted_labels)
        
                    return {
                        "eval_accuracy": accuracy,
                        "true_positives": true_pos,
                        "true_negatives": true_neg,
                        "false_positives": false_pos,
                        "false_negatives": false_neg,
                        }

                eval_metrics = metric_fn(label_ids, predicted_labels)

                if mode == tf.estimator.ModeKeys.TRAIN:
                    return tf.estimator.EstimatorSpec(mode=mode,
                        loss=loss,
                        train_op=train_op)
                else:
                    return tf.estimator.EstimatorSpec(mode=mode,
                        loss=loss,
                        eval_metric_ops=eval_metrics)
            else:
                (predicted_labels, log_probs, output_layer) = self.CreateModel(
                    is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels, BERT_MODEL_HUB)
                predictions = {
                              'probabilities': log_probs,
                              'labels': predicted_labels,
                              'pooled_output': output_layer
                              }
                return tf.estimator.EstimatorSpec(mode, predictions=predictions)

        # Return the actual model function in the closure
        return model_fn

    def InitializeModel(self, BERT_MODEL_HUB):
        batchSize = self.config.training.batchSize
        learningRate = self.config.training.learningRate
        numTrainEpochs = self.config.training.numTrainEpochs
        # Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
        warmUpProportion = self.config.training.warmUpProportion
        # Model configs
        SAVE_CHECKPOINTS_STEPS = 300
        SAVE_SUMMARY_STEPS = 100

        # Compute train and warmup steps from batch size
        self.num_train_steps = int(len(self.input.train_features) / batchSize * numTrainEpochs)
        self.num_warmup_steps = int(self.num_train_steps * warmUpProportion)

        # Specify output directory and number of checkpoint steps to save
        run_config = tf.estimator.RunConfig(
            model_dir= self.config.checkpointPath,
            save_summary_steps=SAVE_SUMMARY_STEPS,
            save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)
        
        #Initializing the model and the estimator
        self.model_fn = self.ModelFnBuilder(
            num_labels=len(self.config.labelList),
            learning_rate=learningRate,
            num_train_steps=self.num_train_steps,
            num_warmup_steps=self.num_warmup_steps,
            BERT_MODEL_HUB = BERT_MODEL_HUB)

        self.estimator = tf.estimator.Estimator(
            model_fn=self.model_fn,
            config=run_config,
            params={"batch_size": batchSize})
        
    def train(self, BERT_MODEL_HUB):
        #Training the model
        print(f'Beginning Training!')
        current_time = datetime.now()
        self.InitializeModel(BERT_MODEL_HUB)
        self.estimator.train(input_fn=self.input.train_input_fn, max_steps=self.num_train_steps)
        print("Training took time ", datetime.now() - current_time)
        
    def evaluate(self):
        #Evaluating the model with Validation set
        self.estimator.evaluate(input_fn=self.input.val_input_fn, steps=None)

    def GetPrediction(self, in_sentences, type_output = "features"):
        # A method to get predictions

        #A list to map the actual labels to the predictions
        labels = self.config.labelList
        input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
        input_features = run_classifier.convert_examples_to_features(input_examples, self.config.labelList, self.config.maxSeqLength, self.input.tokenizer)
        #Predicting the classes 
        predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=self.config.maxSeqLength, is_training=False, drop_remainder=False)
        predictions = self.estimator.predict(predict_input_fn)
        print(predictions)
        if type_output == "features":
            return [prediction['pooled_output'] for _,prediction in enumerate(predictions) ]
        else:
            return ([(sentence, prediction['probabilities'],
                prediction['labels'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)])